# TP06 : Naïve Bayes

Tout le monde connait le théorème de Bayes pour calculer la probabilité conditionnelle d'un évennement $A$ sachant un autre $B$: 
$$ P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

Pour appliquer ce théorème sur un problème d'appentissage automatique, l'idée est simple ; Etant donné une caractéristique $f$ et la sortie $y$ qui peut avoir la classe $c$ : 
- Remplacer $A$ par $y=c$
- Remplacer $B$ par $f$ 
On aura l'équation : 
$$ P(y=c|f) = \frac{P(y=c)P(f|y=c)}{P(f)}$$

On appelle : 
- $P(y=c|f)$ postérieure 
- $P(y=c)$ antérieure
- $P(f|y=c)$ vraisemblance
- $P(f)$ évidence 

Ici, on estime la probablité d'une classe $c$ sachant une caractéristique $f$ en utilisant des données d'entrainement. Maintenant, on veut estimer la probabilité d'une classe $c$ sachant un vecteur de caractéristiques $\overrightarrow{f} = \{f_1, ..., f_L\}$ : 
$$ P(y=c|\overrightarrow{f}) = \frac{P(y=c)P(\overrightarrow{f}|y=c)}{P(f)}$$

Etant donnée plusieurs classes $c_j$, la classe choisie $\hat{c}$ est celle avec la probabilité maximale 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k|\overrightarrow{f})$$
$$\hat{c} = \arg\max\limits_{c_k} \frac{P(y=c_k)P(\overrightarrow{f}|y=c_k)}{P(f)}$$
On supprime l'évidence pour cacher le crime : $P(f)$ ne dépend pas de $c_k$ et elle est postive, donc ça ne va pas affecter la fonction $\max$.
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k)P(\overrightarrow{f}|y=c_k)$$

Pour calculer $P(\overrightarrow{f}|y=c_k)$, on va utiliser une properiété naïve (d'où vient le nom Naive Bayes) : on suppose l'indépendence conditionnelle entre les caractéristiques $f_j$. 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k) \prod\limits_{f_j \in \overrightarrow{f}} P(f_j|y=c_k)$$

Pour éviter la disparition de la probabilité (multiplication et représentation de virgule flottante sur machine), on transforme vers l'espace logarithme.
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$


## Avantages 

Les classifieurs naïfs bayésiens, malgré leur simplicité, ont des points forts:
- Ils ont besoin d'une petite quantité de données d'entrainement.
- Ils sont très rapides par rapport aux autres classifieurs.
- Ils donnent de bons résultats dans le cas de filtrage du courrier indésirable et de classification de documents.

## Limites
Les classifieurs naïfs bayésiens certes sont populaires à cause de leur simplicité. Mais, une telle simplicité vient avec un coût [référence: Spiderman].
- Les probabilités obtenues en utilisant ces classifieurs ne doivent pas être prises au sérieux.
- S'il existe une grande corrélation entre les caractéristiques, ils vont donner une mauvaise performance.
- Dans le cas des caractéristiques continues (prix, surface, etc.), les données doivent suivre la loi normale.


## I- Implémentation

Pour estimer la vraisemblance, il y a plusieurs modèles (lois):
- Loi multinomiale : pour les caracétristiques nominales
- Loi de Bernoulli : lorsqu'on est interressé par l'apparence d'une caractéristique ou non (binaire)
- loi normale : pour les caractéristiques numériques

Dans ce TP, on va implémenter Naive Bayes pour les caractéristiques nominales (loi multinomiale)

### I-1- Les données pour les tests unitaires
Ici, on va utiliser le dataset "jouer" contenant des caractéristiques nominales.

In [1]:
import numpy as np
import pandas as pd 


jouer = pd.read_csv("datasets/jouer.csv")

X_jouer = jouer.iloc[:, :-1].values # Premières colonnes 
Y_jouer = jouer.iloc[:,-1].values # Dernière colonne 

# Afficher le dataset "jouer"
jouer

,temps,temperature,humidite,vent,jouer
0,ensoleile,chaude,haute,non,non
1,ensoleile,chaude,haute,oui,non
2,nuageux,chaude,haute,non,oui
3,pluvieux,douce,haute,non,oui
4,pluvieux,fraiche,normale,non,oui
5,pluvieux,fraiche,normale,oui,non
6,nuageux,fraiche,normale,oui,oui
7,ensoleile,douce,haute,non,non
8,ensoleile,fraiche,normale,non,oui
9,pluvieux,douce,normale,non,oui


### I-2- Estimation de la probabilité antérieure
Etant donné le vecteur de sortie $Y$, on doit calculer la probabilité de chaque classe (différentes valeurs de $Y$)

$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$

La fonction doit retourner un dictionnaire où la clé est le nom de la classe et la valeur est sa probabilité. Voici, un exemple d'un dictionnaire dans Python

In [2]:
# Exemple de dictionnaire dans Python 
d = {}
d["Iris-setosa"] = 0.5
d["Iris-versicolor"] = 0.33
d["Iris-virginica"] = 0.67

for c in d: 
    print("P(" + c + ")= " + str(d[c]))

P(Iris-setosa)= 0.5
P(Iris-versicolor)= 0.33
P(Iris-virginica)= 0.67


In [3]:
# TODO Réaliser la fonction 
def P_c(Y): 
    vals = np.unique(Y)
    resultat = {}
    for i in range(vals.shape[0]):
        resultat[vals[i]] = np.sum(Y == vals[i]) / Y.shape[0]
    return resultat

# Résultat: {'non': 0.35714285714285715, 'oui': 0.6428571428571429}
P_c(Y_jouer)

{'non': 0.35714285714285715, 'oui': 0.6428571428571429}

### I-3- Entrainement  (loi multinomiale)

Notre modèle (notons le par $\theta_{f_j,C}$) doit garder le nombre des différentes valeurs dans une caractéristique $A$ et le nombre de ces valeurs dans chaque classe.

Donc, étant donné un vecteur d'une caractéristique $A$ et un autre des $Y$ respectives, la fonction d'entrainement doit retourner un dictionnaire (notre théta) : 
- la clé est une valeur $a_v$ de $A$ 
- la valeur est un autre dictionnaire : 
   - il doit contenir une clé "_total_" dont la valeur est le nombre d'occurence de $a_v$ dans $A$ 
   - la clé est la classe $c_k$ de $Y$
   - la valeur est le nombre d'occurence de $a_v$ respectives à $c_k$


In [4]:
# TODO Réaliser cette fonction 
# Elle génère théta pour une seule caractéristique
def entrainer_multi_1(A, Y): 
    valsA = np.unique(A)
    valsY = np.unique(Y)
    
    resultat = {}
    for i in range (valsA.shape[0]):
        result = {}
        result['_Total_'] = np.sum(A == valsA[i])
        for j in range (valsY.shape[0]):
            somme = 0
            for k in range (A.shape[0]):
                if (Y[k] == valsY[j] and A[k] == valsA[i]):
                    somme += 1
            result[valsY[j]] = somme
        resultat[valsA[i]] = result
    return resultat

# Résultat 
# {'ensoleile': {'_total_': 5, 'non': 3, 'oui': 2},
# 'nuageux': {'_total_': 4, 'non': 0, 'oui': 4},
# 'pluvieux': {'_total_': 5, 'non': 2, 'oui': 3}}
Theta_jouer_temps = entrainer_multi_1(X_jouer[:, 0], Y_jouer)

Theta_jouer_temps

{'ensoleile': {'_Total_': 5, 'non': 3, 'oui': 2},
 'nuageux': {'_Total_': 4, 'non': 0, 'oui': 4},
 'pluvieux': {'_Total_': 5, 'non': 2, 'oui': 3}}

In [5]:
# La fonction qui entraine Théta sur plusieurs caractéristiques
# Rien à programmer ici
# Notre théta est une liste des dictionnaires;
# chaque dictionnaire contient le théta de la caractéristique respective à la colonne de X
# On ajoute les probabilités antérieures des classes à la fin de résultat
def entrainer_multi(X, Y): 
    resultat = []
    for i in range(X.shape[1]): 
        resultat.append(entrainer_multi_1(X[:, i], Y))
    resultat.append(P_c(Y))
    return resultat

Theta_jouer = entrainer_multi(X_jouer, Y_jouer)

Theta_jouer

[{'ensoleile': {'_Total_': 5, 'non': 3, 'oui': 2},
  'nuageux': {'_Total_': 4, 'non': 0, 'oui': 4},
  'pluvieux': {'_Total_': 5, 'non': 2, 'oui': 3}},
 {'chaude': {'_Total_': 4, 'non': 2, 'oui': 2},
  'douce': {'_Total_': 6, 'non': 2, 'oui': 4},
  'fraiche': {'_Total_': 4, 'non': 1, 'oui': 3}},
 {'haute': {'_Total_': 7, 'non': 4, 'oui': 3},
  'normale': {'_Total_': 7, 'non': 1, 'oui': 6}},
 {'non': {'_Total_': 8, 'non': 2, 'oui': 6},
  'oui': {'_Total_': 6, 'non': 3, 'oui': 3}},
 {'non': 0.35714285714285715, 'oui': 0.6428571428571429}]

### I-4- Estimation de la probabilité de vraissemblance (loi multinomiale)
L'équation pour estimer la vraisemblance 
$$ P(f_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } f_j = v\}|}{|\{y = c_k\}|}$$

Si, dans le dataset de test, on veut calculer la probabilité d'une valeur $v$ qui n'existe pas dans le dataset d'entrainnement ou qui n'existe pas pour une classe donnée, on aura une probabilité nulle. Ici, on doit appliquer une fonction de lissage qui donne une petite probabilité aux données non vues dans l'entrainnement. Le lissage qu'on va utiliser est celui de Lidstone. Lorsque $\alpha = 1$ on l'appelle lissage de Laplace.
$$ P(f_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } f_j = v\}| + \alpha}{|\{y = c_k\}| + \alpha * |V|}$$
Où: 
- $\alpha$ est une valeur donnée 
- $V$ est l'ensemble des différentes valeurs de $f_j$ (le vocabulaire)

In [6]:
# TODO compléter cette fonction
def P_vraiss_multi(Theta_j, v, c, alpha=1.): 
    len_V = len(Theta_j) # La taille du vocabulaire
    nbr_c = 0
    for i in Theta_j.keys() :
        nbr_c += Theta_j[i][c]
    return ((Theta_j[v][c] if (v in Theta_j.keys()) else 0) + alpha) / (nbr_c + alpha * len_V )

# La probabilité de jouer si temps = pluvieux 
# P(temps = pluvieux | jouer=oui) = (nbr(temps=pluvieux et jouer=oui)+alpha)/(nbr(jour=oui) + alpha * nbr_diff(temps)))
# P(temps = pluvieux | jouer=oui) = (3 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = pluvieux | jouer=oui) = 4/12 ==> 0.33333333333333333333333333333333333~

# La probabilité de jouer si temps = neigeux 
# P(temps = neigeux | jouer=oui) = (nbr(temps=neigeux et jouer=oui)+alpha)/(nbr(jouer=oui) + alpha * nbr_diff(temps)))
# P(temps = neigeux | jouer=oui) = (0 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = neigeux | jouer=oui) = 1/13 ==> 0.0833333333333333333333333333333333333~


P_vraiss_multi(Theta_jouer_temps, "pluvieux", "oui"), P_vraiss_multi(Theta_jouer_temps, "neigeux", "oui")

(0.3333333333333333, 0.08333333333333333)

### I-5- Prédiction de la classe (loi multinomiale)
Revenons maintenant à notre équation de prédiction 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

Ici, vous devez prédire un seule échantillon $x$

In [7]:
# TODO compléter ce code
# Pour récupérer le théta de la caractéristique n°0 : Theta[0]
# anter est un booléen, si il est False, on ne compte pas la probabilité antérieure P(y = c_k)
def predire(x, Theta, alpha=1., anter=True): 
    #print(Theta)
    c_opt = "" # la classe optimale
    p_c = Theta[-1] #les classes et leurs probabilités antérieures
    if not anter: # si on ne veut pas ajouter les probabiliés antérieures
        p_c = dict.fromkeys(p_c, 1.) # on définit le tous en 1; log(1) = 0
    max_log_p = np.NINF # - infinity 
    # compléter ici
    for i in p_c.keys() :
        log_p = np.log(p_c[i])
        k = 0
        for j in x :
            log_p += np.log(P_vraiss_multi(Theta[k], j, i, alpha))
            k += 1
        if (log_p > max_log_p):
            c_opt = i
            max_log_p = log_p
    return c_opt, max_log_p

# Résultat: (('oui', -4.102643365036796), ('oui', -3.6608106127577567))
predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer), predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer, anter=False) 

(('oui', -4.102643365036796), ('oui', -3.6608106127577567))

### I-7- Regrouper en une classe (loi multinomiale)

**Rien à programmer ici, il y a une petite analyse**


In [8]:
class NBMultinom(object): 
    
    def __init__(self, alpha=1.): 
        self.alpha = alpha
        
    def entrainer(self, X, Y):
        self.Theta = entrainer_multi(X, Y)
    
    def predire(self, X, anter=True, prob=False): 
        Y_pred = []
        for i in range(len(X)): 
            c, p = predire(X[i,:], self.Theta, alpha=self.alpha, anter=anter)
            if prob:
                Y_pred.append(p)
            else:
                Y_pred.append(c)
        return Y_pred

On va entrainer un modèle en utilisant notre imlémentation avec et sans probabilité antérieure. 
Normalement, on doit tester sur des données non vues (des données qu'on n'a pas utilisé pour l'entrainement). Mais, ici, on va tester sur les mêmes données d'entrainement afin de savoir si le modèle a bien représenté ce dataset ou non (calculer l'erreur) 

In [9]:
notre_modele = NBMultinom()
notre_modele.entrainer(X_jouer, Y_jouer)
Y_notre_ant = notre_modele.predire(X_jouer)
Y_notre_sans_ant = notre_modele.predire(X_jouer, anter=False)

# Ici, ce n'ai pas la peine d'exécuter plusieurs fois
# puisque le résultat sera le même 

# Le rapport de classification
from sklearn.metrics import classification_report

print("Notre modèle avec probabilité antérieure (a priori)")
print(classification_report(Y_notre_ant, Y_jouer))

print("Notre modèle sans probabilité antérieure (a priori)")
print(classification_report(Y_notre_sans_ant, Y_jouer))


Notre modèle avec probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.80      1.00      0.89         4
         oui       1.00      0.90      0.95        10

    accuracy                           0.93        14
   macro avg       0.90      0.95      0.92        14
weighted avg       0.94      0.93      0.93        14

Notre modèle sans probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.80      0.67      0.73         6
         oui       0.78      0.88      0.82         8

    accuracy                           0.79        14
   macro avg       0.79      0.77      0.78        14
weighted avg       0.79      0.79      0.78        14



**Analyser les résultats** <br>
- On remarque que le modèle avec probabilité antérieure donne de meilleurs résultats que modèle sans probabilité antérieure. le premier modèle a une meilleur precison , recall et f1 score que le deuxième modèle. Le premier modèle converge donc mieux que le second car sans la probabilité antérieure, le modèle considère que toutes les caractéristiques ont le même nombre d'observations (échantillons).

## II- Détection de spam 

Ici, on va essayer d'appliquer l'apprentissage automatique sur la détection de spam. 
Chaque message dans le dataset est représenté en utilisant un modèle "Sac à mots" (BoW : Bag of Words).
Dans l'entrainement, on récupère les différents mots qui s'apparaissent dans les messages. 
Chaque mot va être considéré comme une caractéristique. 
Donc, pour chaque message, la valeur de la caractéristique est la fréquence de son mot dans le message. 
Par exemple, si le mot "good" apparait 3 fois dans le message, donc la caractéristique "good" aura la valeur 3 dans ce message.

Notre implémentation n'est pas adéquate pour la nature de ce problème. 
Dans Scikit-learn, le [sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) est similaire à notre implémentation. 
L'algorithme adéquat pour ce type de problème est [sklearn.naive_bayes.MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

Le dataset utilisé est [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset).
Les algorithmes comparés :
- Naive Bayes
- Arbre de décision
- Regression logistique 

### II-1- Préparation de données


In [10]:
messages = pd.read_csv("datasets/spam.csv", encoding="latin-1")
messages = messages.rename(columns={"v1": "classe", "v2": "texte"})
messages = messages.filter(["texte", "classe"])

messages.head()

,texte,classe
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


### II-2- Entrainement et test des modèles sur plusieurs exécutions 

Afin de satisfaire un étudiant qui réclame toujours sur le manque des données, nous avons décidé de comparer les algorithmes sur plusieurs excécutions (runs). 

**Rien à analyser ici**

**P.S.** timeit.default_timer() est dépendante du système d'exploitation. Aussi, elle peut être affectée par d'autre processus en parallèle. 

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import timeit
from sklearn.metrics import precision_score, recall_score

NBR_RUN = 7

temps_train = {
    "naive_bayes" : [],
    "arbre_decision": [],
    "reg_log": []
}

temps_test = {
    "naive_bayes" : [],
    "arbre_decision": [],
    "reg_log": []
}

perf = {
    "naive_bayes_P" : [],
    "arbre_decision_P": [],
    "reg_log_P": [], 
    "naive_bayes_R" : [],
    "arbre_decision_R": [],
    "reg_log_R": []
}


for run in range(NBR_RUN): 
    # prétaitement des données
    msg_train, msg_test, Y_train, Y_test = train_test_split(messages["texte"],messages["classe"],test_size=0.2)
    count_vectorizer = CountVectorizer()
    X_train = count_vectorizer.fit_transform(msg_train)
    X_test = count_vectorizer.transform(msg_test)
    
    # ==================================
    # ENTRAINEMENT 
    # ==================================
    
    #entrainement Naive Bayes
    naive_bayes = MultinomialNB()
    temps_debut = timeit.default_timer()
    naive_bayes.fit(X_train, Y_train)
    temps_train["naive_bayes"].append(timeit.default_timer() - temps_debut)
    
    #entrainement CART
    arbre_decision = DecisionTreeClassifier()
    temps_debut = timeit.default_timer()
    arbre_decision.fit(X_train, Y_train)
    temps_train["arbre_decision"].append(timeit.default_timer() - temps_debut)
    
    #entrainement Régression logitique
    reg_log = LogisticRegression(solver="lbfgs") #solver=sag est plus lent; donc j'ai choisi le plus rapide
    temps_debut = timeit.default_timer()
    reg_log.fit(X_train, Y_train)
    temps_train["reg_log"].append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # TEST 
    # ==================================
    
    #test Naive Bayes
    temps_debut = timeit.default_timer()
    Y_naive_bayes = naive_bayes.predict(X_test)
    temps_test["naive_bayes"].append(timeit.default_timer() - temps_debut)
    
    
    #test CART
    temps_debut = timeit.default_timer()
    Y_arbre_decision = arbre_decision.predict(X_test)
    temps_test["arbre_decision"].append(timeit.default_timer() - temps_debut)
    
    #test Régression logitique
    temps_debut = timeit.default_timer()
    Y_reg_log = reg_log.predict(X_test)
    temps_test["reg_log"].append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # PERFORMANCE 
    # ==================================
    # Ici, on va considérer une classification binaire avec une seule classe "spam" 
    # On ne juge pas le classifieur sur sa capacité de détecter les non spams
    
    perf["naive_bayes_P"].append(precision_score(Y_test, Y_naive_bayes, pos_label="spam"))
    perf["arbre_decision_P"].append(precision_score(Y_test, Y_arbre_decision, pos_label="spam"))
    perf["reg_log_P"].append(precision_score(Y_test, Y_reg_log, pos_label="spam"))
    
    perf["naive_bayes_R"].append(recall_score(Y_test, Y_naive_bayes, pos_label="spam"))
    perf["arbre_decision_R"].append(recall_score(Y_test, Y_arbre_decision, pos_label="spam"))
    perf["reg_log_R"].append(recall_score(Y_test, Y_reg_log, pos_label="spam"))
    
    

temps_train

{'naive_bayes': [0.008581199999753153,
  0.00817430000006425,
  0.008171500000571541,
  0.012063600000146835,
  0.008729700000003504,
  0.014888100000462146,
  0.007862899999963702],
 'arbre_decision': [0.10807889999978215,
  0.09442619999936142,
  0.0900523999998768,
  0.10902809999970486,
  0.12166809999962425,
  0.11890270000003511,
  0.12075439999989612],
 'reg_log': [0.08972889999949984,
  0.059421300000394695,
  0.05611010000029637,
  0.0707167999999001,
  0.05657049999990704,
  0.062007799999264535,
  0.07169219999923371]}

### II-3- Analyse du temps d'apprentissage 

Combien de temps chaque algorithme prend pour entrainer le même dataset d'entrainement


In [59]:
pd.DataFrame(temps_train)

,naive_bayes,arbre_decision,reg_log
0,0.008581,0.108079,0.089729
1,0.008174,0.094426,0.059421
2,0.008172,0.090052,0.056110
3,0.012064,0.109028,0.070717
4,0.008730,0.121668,0.056570
5,0.014888,0.118903,0.062008
6,0.007863,0.120754,0.071692


**Analyser** <br>
- On remarque que le modèle naive bayes a un temps d'apprentissage court par rapport à la régression logistique et au modèle d'arbre de décision. Donc le modèle naive bayes converge plus rapidement que les deux autres modelés (arbre de décision et la régression logistique)
- On remarque aussi que la régression logistique présente de bons temps d'entrainement, contrairement aux arbres de décision qui prennent un temps plus élevé , cela est dû au nombre important de caractéristique.

### II-4- Analyse du temps de test 

Combien de temps chaque algorithme prend pour prédir les classes

In [60]:
pd.DataFrame(temps_test)

,naive_bayes,arbre_decision,reg_log
0,0.000197,0.000399,0.000150
1,0.000194,0.000385,0.000111
2,0.000328,0.000525,0.000148
3,0.000196,0.000351,0.000090
4,0.000362,0.000403,0.000101
5,0.000336,0.000482,0.000141
6,0.000197,0.000366,0.000092


**Analyser** <br>
- On remarque que la régression logistique présente les meilleur résultats car il s'agit d'une régression binaire (spam/non spam) donc un simple calcule d'une valeur en utilisant un vecteur de theta et la comparer avec un seuil.
- Le modèle de naive bayes vient en deuxième position, car pour cela il faut juste calculer la vraisemblance de chacune des caractéristiques et l'antérieure des valeurs des y.
- Les arbres de décision présentent le plus grand temps de test. Cela est dû au nombre de caractéristique(chaque mot présente une) donc l'arbre est trop profond pour le parcourir.

### II-5- Analyse de la performance 

Ici, on compare les modèles en se basant sur leurs capacités à détecter le spam. 
On va utiliser la précision et le rappel.

In [64]:
pd.DataFrame(perf, columns = ["arbre_decision_P", "naive_bayes_P", "reg_log_P", "arbre_decision_R", "naive_bayes_R", "reg_log_R"])

,arbre_decision_P,naive_bayes_P,reg_log_P,arbre_decision_R,naive_bayes_R,reg_log_R
0,0.932836,0.984848,1.000000,0.886525,0.921986,0.886525
1,0.899281,0.977778,0.976744,0.874126,0.923077,0.881119
2,0.887218,0.976378,0.983333,0.855072,0.898551,0.855072
3,0.873134,0.992308,0.961240,0.818182,0.902098,0.867133
4,0.909774,0.968750,0.983607,0.889706,0.911765,0.882353
5,0.904110,0.979452,0.992537,0.862745,0.934641,0.869281
6,0.864865,0.971223,0.984848,0.882759,0.931034,0.896552


**Analyser** <br>
- On remarque que les deux modèles naive bayes et régression logistique ont la même précision contrairement au modèle d'arbre de décision qui est moins précis que les deux autres car l'élagage de l'arbre de décision peut négliger certaines valeurs clés des données d'entraînement, ce qui peut conduire à la précision du modèle.
- On remarque que le modèle naive bayes a un meilleur rappel que les deux autres modèles (régression logistique et les arbres de décisions) car la règle d'indépendance est appliqué sur cette exemple donc le modèle naive bayes est le mieux adapté pour le filtrage du spam .

**Conclusion** <br>
- Le modèle Naive bayes est facile et rapide de prédire la classe de l’ensemble de données de test . Il fonctionne également bien dans la prédiction multi-classes.
- Lorsque l'hypothèse d'indépendance est vérifiée, un classificateur Naive Bayes fonctionne mieux que d'autres modèles comme la régression logistique et vous avez besoin de moins de données d'entraînement.
- Il fonctionne bien en cas de variables d'entrée catégorielles par rapport aux variables numériques. Pour la variable numérique, une distribution normale est supposée (courbe en cloche, qui est une hypothèse forte).